### TP 2: Visión por Computadora

Alumnos: Fabricio Lopretto (a1616), Santiago José Olaciregui (a1611)

Objetivo:
Implementar un detector de máximo enfoque sobre un video aplicando técnicas de análisis espectral similar al que utilizan las cámaras digitales modernas. El video a procesar será: “focus_video.mov”.

2. Cambiar la métrica de enfoque eligiendo uno de los algoritmos explicados en el apéndice de: "Analysis of focus measure operators in shape from focus".

El algoritmo de detección a implementar debe detectar y devolver los puntos de máximo enfoque de manera automática.

El agoritmo elegido es el *Image Contrast.*

Se realiza en dos pasos, el primero calcula el contraste para cada píxel de la imagen realizando la sumatoria de la diferencia absoluta entre la intensidad del píxel en cuestión y sus vecinos en la matriz de enfoque de 3x3 centrada en el píxel en estudio.

$$
C(x, y) = \sum_{i=x-1}^{x+1} \sum_{j=y-1}^{y+1} |I_{xy} - I_{ij}|
$$

Luego, para cada píxel, calcula la métrica (Phi en la fórmula), realizando la sumatoria de los contrastes de los píxeles vecinos.

$$
Phi(x, y) = \sum_{(i, j)=vencindario} C(i, j)
$$

In [13]:
# Instala las librerias necesarias
!pip install opencv-python


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
# Importa las librerias necesarias
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [15]:
# Capta la ruta de la notebook
ruta_notebook = os.getcwd()

# Describe la ruta al video
video_path = ruta_notebook + '/focus_video.mov'

El siguiente script permite ubicar en cada frame la ubicación del píxel donde la métrica es máxima. almacena para todo frame la posición del píxel y el valor máximo de la métrica en el mismo. Además imprime esta información por pantalla.

In [16]:
# Abre el video
cap = cv2.VideoCapture(video_path)

# Verifica si se abrio correctamente
if not cap.isOpened():
    print("Error al abrir el archivo de video.")

# Obtiene el numero de frames totales
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Define el tamano del vecindario para Phi (aqui se usa 3x3)
vecindario_size = 3
offset = vecindario_size // 2

# Inicializa la matriz de 3 dimensiones para almacenar la posicion maxima de Phi
# Cada entrada en la matriz sera (frame_num, [x, y])
max_phi_positions = np.zeros((total_frames, 2), dtype=int)

# Procesa cada frame
frame_num = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convierte a escala de grises
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    alto, ancho = gray_frame.shape

    # Inicializa los valores de contraste local y global
    contraste_local = np.zeros_like(gray_frame, dtype=np.float32)
    phi = np.zeros_like(gray_frame, dtype=np.float32)

    # Calcula el contraste local C(x, y)
    for x in range(1, alto-1):
        for y in range(1, ancho-1):
            suma_contraste = 0
            for i in range(x-1, x+2):
                for j in range(y-1, y+2):
                    suma_contraste += abs(int(gray_frame[x, y]) - int(gray_frame[i, j]))
            contraste_local[x, y] = suma_contraste

    # Calcula la metrica Phi(x, y) en funcion del vecindario
    for x in range(offset, alto-offset):
        for y in range(offset, ancho-offset):
            phi[x, y] = np.sum(contraste_local[x-offset:x+offset+1, y-offset:y+offset+1])

    # Encuentra el valor maximo de Phi y su ubicacion
    max_phi_value = np.max(phi)
    max_phi_location = np.unravel_index(np.argmax(phi), phi.shape)

    # Guarda la ubicacion del pixel en la matriz tridimensional
    max_phi_positions[frame_num] = max_phi_location

    # Imprime la ubicacion del pixel donde Phi es maximo
    print(f"Frame {frame_num}: Máxima Phi en la ubicación {max_phi_location} con valor {max_phi_value}")
    
    frame_num += 1

# Libera el video
cap.release()

Frame 0: Máxima Phi en la ubicación (116, 405) con valor 1107.0
Frame 1: Máxima Phi en la ubicación (115, 405) con valor 1130.0
Frame 2: Máxima Phi en la ubicación (114, 405) con valor 1127.0
Frame 3: Máxima Phi en la ubicación (113, 405) con valor 1127.0
Frame 4: Máxima Phi en la ubicación (112, 405) con valor 1157.0
Frame 5: Máxima Phi en la ubicación (112, 407) con valor 1147.0
Frame 6: Máxima Phi en la ubicación (111, 407) con valor 1161.0
Frame 7: Máxima Phi en la ubicación (110, 407) con valor 1151.0
Frame 8: Máxima Phi en la ubicación (109, 407) con valor 1194.0
Frame 9: Máxima Phi en la ubicación (108, 406) con valor 1193.0
Frame 10: Máxima Phi en la ubicación (108, 407) con valor 1193.0
Frame 11: Máxima Phi en la ubicación (109, 409) con valor 1099.0
Frame 12: Máxima Phi en la ubicación (107, 407) con valor 1184.0
Frame 13: Máxima Phi en la ubicación (108, 409) con valor 1121.0
Frame 14: Máxima Phi en la ubicación (108, 410) con valor 1164.0
Frame 15: Máxima Phi en la ubicació

El siguiente script usa la matriz donde se guarda la posición del máximo de nitidez por frame, y se la utiliza para plasmar la ubicación con un punto verde de estos en el video:

In [12]:
# Abre el video
cap = cv2.VideoCapture(video_path)

# Verifica si se abrio correctamente
if not cap.isOpened():
    print("Error al abrir el archivo de video.")

# Obtiene propiedades del video original
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Crea un objeto VideoWriter para el nuevo video
output_path = ruta_notebook + '/output_video_with_max_phi.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codificación para MP4
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Procesa el video frame por frame
frame_num = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Obtiene la posicion maxima de Phi para el frame actual
    max_position = max_phi_positions[frame_num]
    
    # Dibuja un punto verde en la posicion maxima de Phi
    cv2.circle(frame, (max_position[1], max_position[0]), radius=5, color=(0, 255, 0), thickness=-1)
    
    # Escribe el frame modificado en el nuevo video
    out.write(frame)
    
    frame_num += 1

# Libera los recursos
cap.release()
out.release()

# Da aviso de la creacion del video
print(f"Nuevo video generado y guardado en {output_path}")

Nuevo video generado y guardado en c:\Users\fabri\Documents\Posgrado\VpC1\entregables\TP2/output_video_with_max_phi.mp4


### Conclusiones:

A partir de la impresión por pantalla de la ubicación del píxel con el máximo valor de la métrica por frame, se ve que la ubicación permanece aproximadamente estable en gran parte del video, para luego migrar a la parte inferior derecha cerca del borde. Esto tiene sentido ddo que al ver el video hay un desfasaje entre la puesta en foco del centro de la imagen y los bordes.

#### Bibliografia

S. Pertuz and D. Puig. 2012. Analysis of focus measure operators in shapefrom-focus. DOI: 10.1016/j.patcog.2012.11.011